In [1]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read data

member = pd.read_csv('/Users/maggietang/Downloads/Member Details.csv')
visit = pd.read_csv('/Users/maggietang/Downloads/Visit Dataset.csv')
outcome = pd.read_csv('/Users/maggietang/Downloads/Program Outcomes.csv')
interaction = pd.read_csv('/Users/maggietang/Downloads/Interactions Dataset.csv')

In [3]:
# datasets shape

member.shape, visit.shape, outcome.shape, interaction.shape

((3218, 41), (24882, 57), (3218, 12), (90645, 6))

In [4]:
member.isna().sum()
#visit.isna().sum(), outcome.isna().sum(), interaction.isna().sum()

patient_id                                     0
estimated_delivery_date                     1504
gravida                                     1068
parity                                      1349
marital_status                              1549
date_of_birth                                  0
zip_3                                          0
zip_income_median                             26
zip_income_mean                               23
education_level                             2428
is_nulliparous                                 0
has_pcp_at_enrollment                          0
history_of_opioids_at_enrollment               0
history_of_bh_at_enrollment                    0
asthma_at_enrollment                           0
diabetes_at_enrollment                         0
history_of_gd_at_enrollment                    0
hypertension_at_enrollment                     0
history_of_preterm_at_enrollment               0
active_nicotine_use_at_enrollment              0
has_financial_barrie

In [5]:
member.describe()
# visit.describe(), outcome.describe(), interaction.describe()

,gravida,parity,marital_status,zip_3,zip_income_median,zip_income_mean,education_level,has_pcp_at_enrollment,history_of_opioids_at_enrollment,asthma_at_enrollment,...,chronological_age_weeks_at_registration,race_arab,race_black_aa,race_egyptian,race_others,race_white_caucasian,ethnicity_hispanic_latino,ethnicity_patient_declined,preferred_language_declined,preferred_language_english
count,2150.000000,1869.000000,1669.000000,3218.000000,3192.000000,3195.000000,790.000000,3218.000000,3218.000000,3218.000000,...,192.000000,3199.000000,3199.000000,3199.000000,3199.000000,3199.000000,3197.000000,3197.000000,3208.000000,3208.000000
mean,2.849302,1.739968,0.272618,187.325047,43685.113409,60503.567449,2.487342,0.964264,0.378496,0.501243,...,7.109375,0.000313,0.497968,0.000313,0.266021,0.235386,0.098843,0.422584,0.004676,0.977556
std,2.120956,1.488288,0.445440,20.221604,17802.051496,22102.666398,0.746240,0.185661,0.485088,0.500076,...,11.611671,0.017680,0.500074,0.017680,0.441944,0.424306,0.298497,0.494048,0.068230,0.148145
min,0.000000,0.000000,0.000000,70.000000,18285.000000,29630.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,191.000000,29308.000000,45325.000000,2.000000,1.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2.000000,1.000000,0.000000,191.000000,38087.000000,53134.000000,3.000000,1.000000,0.000000,1.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,4.000000,2.000000,1.000000,191.000000,54178.000000,68789.000000,3.000000,1.000000,1.000000,1.000000,...,8.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,23.000000,14.000000,1.000000,198.000000,141426.000000,328825.000000,3.000000,1.000000,1.000000,1.000000,...,56.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
# Count total interactions per member

interaction['total_interactions'] = (
    interaction[['interaction_type_EMAIL',
                 'interaction_type_SMS',
                 'interaction_type_VISIT',
                 'interaction_type_VOICE_CALL']].sum(axis=1)
)

member_interactions = (
    interaction.groupby('patient_id')['total_interactions']
               .sum()
               .reset_index()
               .rename(columns={'total_interactions': 'engagement_score'})
)

In [7]:
# Count total visits per member

visit['total_visits'] = (
    visit[['visit_status_completed',
           'visit_status_canceled',
           'visit_status_no_show',
           'visit_status_scheduled']].sum(axis=1)
)

member_visits = (
    visit.groupby('patient_id')['total_visits']
         .sum()
         .reset_index()
)


In [12]:
# Build engagement dataset and fill missing metrics

member['program_start_date'] = pd.to_datetime(member['program_start_date'])
member['program_end_date'] = pd.to_datetime(member['program_end_date'])

engagement = (
    member[['patient_id', 'program_start_date', 'program_end_date']]
    .merge(member_interactions, on='patient_id', how='left')
    .merge(member_visits, on='patient_id', how='left')
)

engagement.fillna({'engagement_score': 0, 'total_visits': 0}, inplace=True)


In [13]:
engagement.isna().sum()

# need to fill missing values in program_end_date

patient_id               0
program_start_date       0
program_end_date      2648
engagement_score         0
total_visits             0
dtype: int64

In [14]:
# Calculate median time in program for members with valid end dates

valid_durations = (
    engagement.dropna(subset=['program_end_date'])
    .assign(duration=lambda df: (df['program_end_date'] - df['program_start_date']).dt.days)
)

median_duration = valid_durations['duration'].median()

# Fill missing program_end_date with start_date + median program duration
engagement['program_end_date'] = engagement['program_end_date'].fillna(
    engagement['program_start_date'] + pd.to_timedelta(median_duration, unit='D')
)


In [15]:
# Convert program dates to datetime and calculate each member's retention duration in days.

#engagement['program_start_date'] = pd.to_datetime(engagement['program_start_date'])
#engagement['program_end_date'] = pd.to_datetime(engagement['program_end_date'])

engagement['retention_days'] = (
    engagement['program_end_date'] - engagement['program_start_date']
).dt.days


In [16]:
# Compute key engagement metrics including interactions, visits, and average retention.

eng_summary = {
    'Avg Interactions per Member': float(engagement['engagement_score'].mean()),
    'Median Interactions': float(engagement['engagement_score'].median()),
    'Avg Visits per Member': float(engagement['total_visits'].mean()),
    'Avg Retention Days': float(engagement['retention_days'].mean())
}

eng_summary

{'Avg Interactions per Member': 32.75699192044748,
 'Median Interactions': 17.0,
 'Avg Visits per Member': 9.213486637663145,
 'Avg Retention Days': 211.5074580484773}

In [17]:
engagement.describe()

,program_start_date,program_end_date,engagement_score,total_visits,retention_days
count,3218,3218,3218.000000,3218.000000,3218.000000
mean,2023-05-08 21:56:02.834058496,2023-12-06 19:59:15.251709440,32.756992,9.213487,211.507458
min,2021-10-20 00:00:00,2022-05-09 12:00:00,0.000000,0.000000,-5.000000
25%,2022-04-07 06:00:00,2022-11-01 12:00:00,7.000000,2.000000,201.000000
50%,2023-04-18 00:00:00,2023-12-05 00:00:00,17.000000,5.000000,201.000000
75%,2024-04-24 00:00:00,2024-12-31 00:00:00,48.750000,13.000000,201.000000
max,2025-04-23 00:00:00,2025-11-10 12:00:00,279.000000,86.000000,1248.000000
std,NaN,NaN,37.369713,10.717533,116.971618


In [18]:
# best cutoff based on above stats, adjust based on business needs

bins = [0, 7, 17, 49, 1000]
labels = ['Low', 'Moderate', 'High', 'Very High']

engagement['engagement_level'] = pd.cut(
    engagement['engagement_score'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

engagement['engagement_level'].value_counts()

engagement_level
Low          916
High         815
Very High    789
Moderate     698
Name: count, dtype: int64

In [19]:
engagement[['engagement_score', 'retention_days']].corr()


,engagement_score,retention_days
engagement_score,1.000000,0.233399
retention_days,0.233399,1.000000
